In [39]:
import random
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import joblib


target_scaler = joblib.load("target_scaler.pkl")
final_model = load_model("final_model.h5")

In [30]:
df_model = pd.read_csv('model_data.csv')

In [31]:
TARGET_COL = "sales_revenue_with_tax_clip"

num_cols = [
    "payment_total_tip",
    "payment_count_box",
    "order_duration_seconds_box",
    "prev_week_sales",
    "prev_2week_sales",
    "prev_3week_sales",
    "moving_avg_4weeks",
    "is_holiday"
]

cat_cols = [
    "concept",
    "city",
    "venue_xref_id"
]

In [32]:
df_model = df_model.sort_values(["venue_xref_id", "year_week"]).reset_index(drop=True)

In [85]:
def create_sequences(grouped_df, num_cols, cat_cols, target_col, window_size=4):
    X_num_all, X_cat_all, y_all = [], [], []
    
    for v_id in grouped_df["venue_xref_id"].unique():
        sub = grouped_df[grouped_df["venue_xref_id"] == v_id].reset_index(drop=True)
        n = len(sub)
        if n <= window_size:
            continue 
        
        for i in range(n - window_size):
            X_num_seq = sub[num_cols].iloc[i : i + window_size].values
            X_cat_seq = sub[cat_cols].iloc[i : i + window_size].values
            y_val = sub[target_col].iloc[i + window_size]
            
            X_num_all.append(X_num_seq)
            X_cat_all.append(X_cat_seq)
            y_all.append(y_val)
    
    X_num_all = np.array(X_num_all)  
    X_cat_all = np.array(X_cat_all) 
    y_all = np.array(y_all)  
    return X_num_all, X_cat_all, y_all

window_size = 4
X_num, X_cat, y = create_sequences(df_model, num_cols=num_cols, cat_cols=cat_cols, target_col=TARGET_COL, window_size=window_size)

X_concept = X_cat[:, :, 0]
X_city    = X_cat[:, :, 1]
X_venue   = X_cat[:, :, 2]

In [86]:
unique_venues = df_model["venue_xref_id"].unique()
selected_venue = random.choice(unique_venues)
print("Selected venue_xref_id:", selected_venue)

selected_indices = [i for i in range(len(y)) if X_cat[i, 0, 2] == selected_venue]

Selected venue_xref_id: 28


In [90]:
if len(selected_indices) == 0:
    print("No sequence exists.")
else:
    idx = random.choice(selected_indices)
    print("Sequence index:", idx)
    
    X_concept_sample = X_cat[idx:idx+1, :, 0]  
    X_city_sample    = X_cat[idx:idx+1, :, 1] 
    X_venue_sample   = X_cat[idx:idx+1, :, 2] 
    X_num_sample     = X_num[idx:idx+1] 

    test_sample = {
        "concept_input": X_concept_sample,
        "city_input": X_city_sample,
        "venue_input": X_venue_sample,
        "num_input": X_num_sample
    }
    
    y_pred_scaled = final_model.predict(test_sample)
    y_pred_scaled = y_pred_scaled.flatten()[0]

    actual_value_scaled = y[idx]
    
    y_pred_original = target_scaler.inverse_transform(np.array([y_pred_scaled]).reshape(-1, 1)).flatten()[0]
    y_actual_original = target_scaler.inverse_transform(np.array([actual_value_scaled]).reshape(-1, 1)).flatten()[0]
    
    print("Predicted revenue:", y_pred_original)
    print("Actual revenue:", y_actual_original)
    print("Absolute difference:" + str(abs(y_actual_original - y_pred_original).round(2)))

Sequence index: 588
1/1 [==============================] - 0s 90ms/step
Predicted revenue: 9143.696
Actual revenue: 10869.85
Absolute difference:1726.15
